# KeyBERT Mechanism

* [Keyword Extraction with BERT](https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea)

The author of KeyBERT explains the keyword extraction using BERT (Sentence BERT).

> Although there are many great papers and solutions out there that use BERT-embeddings (e.g., 1, 2, 3, ), I could not find a simple and easy-to-use BERT-based solution. Instead, I decide to create KeyBERT a minimal and easy-to-use keyword extraction technique that leverages BERT embeddings. Now, the main topic of this article will not be the use of KeyBERT but a tutorial on how to use BERT to create your own keyword extraction model.

In [14]:
import itertools
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [5]:
import sklearn
sklearn.__version__

'1.2.1'

In [2]:
doc = """
Supervised learning is the machine learning task of 
learning a function that maps an input to an output based 
on example input-output pairs.[1] It infers a function 
from labeled training data consisting of a set of 
training examples.[2] In supervised learning, each 
example is a pair consisting of an input object 
(typically a vector) and a desired output value (also 
called the supervisory signal). A supervised learning 
algorithm analyzes the training data and produces an 
inferred function, which can be used for mapping new 
examples. An optimal scenario will allow for the algorithm 
to correctly determine the class labels for unseen 
instances. This requires the learning algorithm to  
generalize from the training data to unseen situations 
in a 'reasonable' way (see inductive bias).
"""

In [28]:
n_gram_range = (1, 2)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

In [29]:
candidates[:10]

array(['algorithm', 'algorithm analyzes', 'algorithm correctly',
       'algorithm generalize', 'allow', 'allow algorithm', 'analyzes',
       'analyzes training', 'based', 'based example'], dtype=object)

# Embedding

In [30]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [31]:
candidate_embeddings.shape

(117, 768)

# Document to words similarities

In [32]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [33]:
keywords

['algorithm analyzes',
 'supervised learning',
 'machine learning',
 'learning machine',
 'learning algorithm']

# Diversification

1. Max Sum Similarity
2. Maximal Marginal Relevance


## Max Sum Similarity

> maximize the candidate similarity to the document whilst minimizing the similarity between candidates.
> <img src="./image/max_sum_similarity.png" align="left" width=700/>

In [34]:
def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [35]:
max_sum_sim(
    doc_embedding,
    candidate_embeddings,
    candidates,
    top_n=5,
    nr_candidates=10
)

['training data',
 'algorithm generalize',
 'supervised learning',
 'machine learning',
 'learning algorithm']

## Maximal Marginal Relevance (MMR)

> MMR tries to minimize redundancy and maximize the diversity of results in text summarization tasks. Fortunately, a keyword extraction algorithm called EmbedRank has implemented a version of MMR that allows us to use it for diversifying our keywords/keyphrases.

> <img src="./image/mmr.png" align="left" width=700/>

In [36]:
def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [37]:
mmr(
    doc_embedding,
    candidate_embeddings,
    candidates,
    top_n=5,
    diversity=0.2
)

['learning algorithm',
 'machine learning',
 'supervised learning',
 'learning machine',
 'algorithm generalize']